In [1]:
#|default_exp resample_img

# Resampling hyperspectral data

In [2]:
#|hide
from nbdev.showdoc import *

d:\git\hyperspectral_tools\.venv\Lib\site-packages\nbdev\showdoc.py:28: SyntaxWarning: invalid escape sequence '\{'
  for c in '|^': s = re.sub(rf'\\?\{c}', f'\{c}', s)


In [3]:
#|hide
#|export

import numpy as np
import pandas as pd
from spectral import *